# Analyzing NYPD Historical Crime Data: Impact on academic performance on SAT

## Abstract 

This is a breakdown of every criminal complaint report filed in NYC by the NYPD in 2011. This data is manually extracted every quarter and reviewed by the Office of Management Analysis and Planning. Each record represents a criminal complaint in NYC and includes information abot the type of crime, the location and time of enforcement. This data can be used by the public to explore the nature of criminal activity in conjunction with the Safety/Respect Scores and SAT performance. 

## Methods 

This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) from 2006 to the end of last year (2017). For additional details, please see the [data dictionary](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i). 

In [2]:
#Libraries used for analysis and visualizations 

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
import folium
import branca.colormap as cm
import cufflinks as cf  # interactive plotting made easy (pandas style)

# Using plotly + cufflinks in offline mode
from plotly.offline import iplot, init_notebook_mode
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [4]:
crime_2011 = pd.read_csv('schools/NYPD_Complaint_Data_Historic_2011.csv', index_col = 'CMPLNT_NUM')
crime_2011.head()

,CMPLNT_FR_DT,CMPLNT_TO_DT,RPT_DT,OFNS_DESC,PD_CD,LAW_CAT_CD,BORO_NM,Latitude,Longitude,PATROL_BORO,VIC_AGE_GROUP
CMPLNT_NUM,,,,,,,,,,,
519234712,1/1/2011,1/1/2011,1/21/2019,SEX CRIMES,177.0,FELONY,BROOKLYN,40.648650,-73.950336,PATROL BORO BKLYN SOUTH,<18
495207193,1/1/2011,2/13/2019,2/13/2019,PETIT LARCENY,338.0,MISDEMEANOR,MANHATTAN,40.862255,-73.926297,PATROL BORO MAN NORTH,45-64
527650130,1/1/2011,NaN,2/8/2019,SEX CRIMES,177.0,FELONY,BRONX,40.816392,-73.895296,PATROL BORO BRONX,<18
512891731,1/1/2011,NaN,3/7/2019,FRAUDS,718.0,MISDEMEANOR,BROOKLYN,40.633193,-73.941884,PATROL BORO BKLYN SOUTH,45-64
234370701,1/1/2011,NaN,4/5/2019,RAPE,157.0,FELONY,QUEENS,40.745253,-73.870063,PATROL BORO QUEENS NORTH,<18


In [8]:
crime_2011.isnull().sum()

CMPLNT_FR_DT          0
CMPLNT_TO_DT     134235
RPT_DT                0
OFNS_DESC          1625
PD_CD               494
LAW_CAT_CD            0
BORO_NM             499
Latitude             54
Longitude            54
PATROL_BORO         494
VIC_AGE_GROUP    170162
dtype: int64

In [10]:
crime_2011.shape

(498734, 11)

In [17]:
crime_df = crime_2011[crime_2011['BORO_NM'].notna()]

crime_df.head()

,CMPLNT_FR_DT,CMPLNT_TO_DT,RPT_DT,OFNS_DESC,PD_CD,LAW_CAT_CD,BORO_NM,Latitude,Longitude,PATROL_BORO,VIC_AGE_GROUP
CMPLNT_NUM,,,,,,,,,,,
519234712,1/1/2011,1/1/2011,1/21/2019,SEX CRIMES,177.0,FELONY,BROOKLYN,40.648650,-73.950336,PATROL BORO BKLYN SOUTH,<18
495207193,1/1/2011,2/13/2019,2/13/2019,PETIT LARCENY,338.0,MISDEMEANOR,MANHATTAN,40.862255,-73.926297,PATROL BORO MAN NORTH,45-64
527650130,1/1/2011,NaN,2/8/2019,SEX CRIMES,177.0,FELONY,BRONX,40.816392,-73.895296,PATROL BORO BRONX,<18
512891731,1/1/2011,NaN,3/7/2019,FRAUDS,718.0,MISDEMEANOR,BROOKLYN,40.633193,-73.941884,PATROL BORO BKLYN SOUTH,45-64
234370701,1/1/2011,NaN,4/5/2019,RAPE,157.0,FELONY,QUEENS,40.745253,-73.870063,PATROL BORO QUEENS NORTH,<18
